In [ ]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import scala.util.{Failure, Success, Try}
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession

In [ ]:
val spark = (
    SparkSession
    .builder()
    .appName("KospiVolatility")
    .config("spark.cores.max", 4)
    .config("spark.executor.cores", 4)
    .config("spark.executor.memory", "36g")
    .getOrCreate()
    )

In [ ]:
val mongoUrl = "mongodb+srv://xxxxxxxxxxxxxxxxxxxxxx/"

val kospiVolatility = (
    spark.read.format("mongodb")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "KospiVolatility").load()
    .select("updateDate", "closingPrice")
    )

In [ ]:
val partition = Window.orderBy("updateDate")
val w_252 = partition.rowsBetween(-252, 0)

val kospiVolatilityCalcurate = (
    kospiVolatility
    .withColumn("avg252", avg(col("closingPrice")).over(w_252))
    .withColumn("volatilityThreshold", col("avg252") / col("closingPrice"))
    .withColumn("volatilityThreshold", when(col("volatilityThreshold") < 1, col("volatilityThreshold")).otherwise(1))
    .where(col("updateDate") > "20230630")
    .select("updateDate", "volatilityThreshold")
    )

In [ ]:
kospiVolatilityCalcurate.show(200)

In [ ]:
"""
(
    kospiVolatilityCalcurate 
    .write.format("mongodb")
    .mode("append")
    .option("upsertDocument", "true")
    .option("idFieldList", "updateDate")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "KospiVolatilityCalcurate")
    .save()
    )
    """

In [ ]:
spark.stop()